In [3]:
import pandas as pd
from cleaning_function import *
data = pd.read_csv('cvrm_master.csv')

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6600/187753911.py:3: DtypeWarning: Columns (1697,1698) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cvrm_master.csv')


In [4]:
data = drop_null_columns(data, 0.6)
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
from sklearn.mixture import GaussianMixture

df = data
def gmm_imputation(df, n_components=3):
    """Impute missing values using Gaussian Mixture Model (GMM)."""
    df_imputed = df.copy()
    for col in df.columns:
        if df[col].isna().any():  # Only apply to columns with missing values
            # Fit GMM on non-missing values
            gmm = GaussianMixture(n_components=n_components, random_state=42)
            gmm.fit(df[[col]].dropna())
            
            # Sample missing values from the GMM
            missing_indices = df[col].isna()
            df_imputed.loc[missing_indices, col] = gmm.sample(missing_indices.sum())[0].flatten()
    return df_imputed

# Apply GMM imputation
df_imputed = gmm_imputation(df)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than

In [7]:
df_imputed.isnull().sum().sum()

0